In [1]:
import pandas as pd
from metaflow import Flow, Run, Step

## Collect results of experiments from Metaflow

In [2]:
def get_results(run_id: int) -> pd.DataFrame:
    try:
        # Try loading the final results if run has finished
        results_for_run = Run(f"ForecastEvaluation/{run_id}").data.results_full
    except (KeyError, AttributeError):
        # Manually collecting results for run if some jobs are still in progress
        results_list = []
        for t in Step(f"ForecastEvaluation/{run_id}/evaluate_dataset").tasks():
            try:
                results_list.append(t.data.results)
            except KeyError:
                pass
        results_for_run = pd.concat(results_list)
    return results_for_run

### Option 1: Provide IDs of runs for each model

In [3]:
save_to_disk = False

In [4]:
# Make sure to replace these with your run IDs!
results_all = pd.concat([
    get_results(1712079941097970),  # SeasonalNaive
    get_results(1712079795572065),  # StatisticalEnsemble
    get_results(1712080010851589),  # chronos_mini
    get_results(1712081461874960),  # chronos_large
])
if save_to_disk:
    results_all.to_csv("results/results_all.csv", index=False)

### Option 2: Collect results from the most recent runs

In [5]:
results_all = []
for run in list(Flow("ForecastEvaluation").runs())[:5]:
    results_all.append(get_results(run.id))
results_all = pd.concat(results_all).dropna(subset="value").drop_duplicates(["dataset", "model", "metric"])
if save_to_disk:
    results_all.to_csv("results/results_all.csv", index=False)

### Option 3: Load results from disk

In [6]:
results_all = pd.read_csv("results/results_all.csv")

## Combine the results into a table

In [7]:
table = results_all.set_index(["dataset", "metric", "model"]).unstack(level=1).unstack(1).round(3).droplevel(0, axis=1)

In [8]:
# StatisticalEnsemble takes >24 hours to forecast on `ett_small_15min`, so the result is missing for this dataset.
dataset_order = [
    "australian_electricity_demand", 
    "car_parts_without_missing",
    "cif_2016", 
    "covid_deaths", 
    "dominick", 
    "ercot", 
    "ett_small_15min",
    "ett_small_1h", 
    "exchange_rate", 
    "fred_md", 
    "hospital", 
    "m5",
    "nn5_daily_without_missing", 
    "nn5_weekly", 
    "traffic", 
    "weather",
    "m1_monthly",
    "m1_quarterly", 
    "m1_yearly", 
    "m3_monthly", 
    "m3_other", 
    "m3_quarterly",
    "m3_yearly", 
    "m4_quarterly", 
    "m4_yearly", 
    "tourism_monthly",
    "tourism_quarterly", 
    "tourism_yearly", 
]
table = table.reindex(dataset_order)
table

metric                                 mase                      \
model                         SeasonalNaive StatisticalEnsemble   
dataset                                                           
australian_electricity_demand         1.340               1.215   
car_parts_without_missing             1.120               1.050   
cif_2016                              1.289               0.918   
covid_deaths                          7.762               5.246   
dominick                              0.828               0.848   
ercot                                 0.761               1.979   
ett_small_15min                       0.768                 NaN   
ett_small_1h                          0.932               1.003   
exchange_rate                         1.524               1.429   
fred_md                               1.101               0.489   
hospital                              0.921               0.748   
m5                                    1.867               1.638   
nn5_daily_without_missing             1.011               0.844   
nn5_weekly                            1.063               0.974   
traffic                               1.785               1.163   
weather                               0.755               0.675   
m1_monthly                            1.314               1.034   
m1_quarterly                          2.078               1.594   
m1_yearly                             4.894               3.574   
m3_monthly                            1.146               0.826   
m3_other                              1.474               0.727   
m3_quarterly                          1.425               1.164   
m3_yearly                             3.172               2.704   
m4_quarterly                          1.602               1.142   
m4_yearly                             3.966               3.002   
tourism_monthly                       1.631               1.441   
tourism_quarterly                     1.699               1.506   
tourism_yearly                        3.552               3.277   

metric                                                                        \
model                         amazon/chronos-t5-large amazon/chronos-t5-mini   
dataset                                                                        
australian_electricity_demand                   1.184                  0.882   
car_parts_without_missing                       0.807                  0.803   
cif_2016                                        0.986                  1.025   
covid_deaths                                    6.540                  6.555   
dominick                                        0.786                  0.782   
ercot                                           0.578                  0.585   
ett_small_15min                                 0.714                  0.739   
ett_small_1h                                    0.737                  0.805   
exchange_rate                                   1.882                  2.118   
fred_md                                         0.571                  0.564   
hospital                                        0.810                  0.815   
m5                                              1.433                  1.432   
nn5_daily_without_missing                       0.824                  0.900   
nn5_weekly                                      0.929                  0.933   
traffic                                         0.644                  0.640   
weather                                         0.565                  0.598   
m1_monthly                                      1.089                  1.193   
m1_quarterly                                    1.730                  1.799   
m1_yearly                                       4.329                  5.160   
m3_monthly                                      0.854                  0.899   
m3_other                                        0.845                  0.869   
m3

### Compute average performance using geometric mean
1. For each dataset and each model, we compute the **relative score** by dividing the model score by the score of the baseline (SeasonalNaive). This makes the scores comparable across datasets.
2. We aggregate the relative scores of each model across all datasets by taking the **geometric mean** (as recommended by [Fleming & Wallace](https://dl.acm.org/doi/10.1145/5666.5673)).

In [9]:
from scipy.stats import gmean

results = []
for metric in ["mase", "scaled_crps", "smape"]:
    scaled = table[metric].divide(table[metric]["SeasonalNaive"], axis=0).fillna(1.0)
    agg = pd.concat({metric: scaled.apply(gmean)})
    results.append(agg)
pd.concat(results).round(3).to_frame().T

mase                                              \
model SeasonalNaive StatisticalEnsemble amazon/chronos-t5-large   
0               1.0               0.836                    0.81   

                               scaled_crps                      \
model amazon/chronos-t5-mini SeasonalNaive StatisticalEnsemble   
0                      0.845           1.0               0.505   

                                                             smape  \
model amazon/chronos-t5-large amazon/chronos-t5-mini SeasonalNaive   
0                       0.472                  0.485           1.0   

                                                                          
model StatisticalEnsemble amazon/chronos-t5-large amazon/chronos-t5-mini  
0                   0.987                   1.034                  1.085

## Create LaTeX table

In [10]:
full_df = []
for metric in ["mase", "scaled_crps", "smape", "time"]:
    tab = table[metric][["StatisticalEnsemble", "amazon/chronos-t5-large", "amazon/chronos-t5-mini", "SeasonalNaive"]]
    if metric == "time":
        tab = tab.round(1)
    tab = tab.rename(columns={"amazon/chronos-t5-large": "Chronos (large)", "amazon/chronos-t5-mini": "Chronos (mini)"})
    formatted_df = []
    for dataset, row in tab.iterrows():
        first, second = row.nsmallest(2).index
        d = 1 if metric == "time" else 3
        row = row.apply(lambda x: f"{x:.{d}f}")
        row.loc[first] = r"\textbf{" + row.loc[first] + "}"
        row.loc[second] = r"\underline{" + row.loc[second] + "}"
        formatted_df.append(row.to_frame().T)
    formatted_df = pd.concat(formatted_df)
    formatted_df.columns = pd.MultiIndex.from_product([[metric], formatted_df.columns])
    full_df.append(formatted_df)
full_df = pd.concat(full_df, axis=1)
full_df.index = [x.replace("_", "-") for x in full_df.index]

In [11]:
print(full_df.style.to_latex().replace("nan", "N/A"))

\begin{tabular}{lllllllllllllllll}
 & \multicolumn{4}{r}{mase} & \multicolumn{4}{r}{scaled_crps} & \multicolumn{4}{r}{smape} & \multicolumn{4}{r}{time} \\
model & StatisticalEnsemble & Chronos (large) & Chronos (mini) & SeasonalNaive & StatisticalEnsemble & Chronos (large) & Chronos (mini) & SeasonalNaive & StatisticalEnsemble & Chronos (large) & Chronos (mini) & SeasonalNaive & StatisticalEnsemble & Chronos (large) & Chronos (mini) & SeasonalNaive \\
australian-electricity-demand & 1.215 & \underline{1.184} & \textbf{0.882} & 1.340 & 0.058 & \underline{0.054} & \textbf{0.042} & 0.098 & 0.055 & \underline{0.051} & \textbf{0.040} & 0.059 & 12602.9 & 4.7 & \underline{2.1} & \textbf{0.3} \\
car-parts-without-missing & 1.050 & \underline{0.807} & \textbf{0.803} & 1.120 & 1.132 & \underline{1.059} & \textbf{1.022} & 2.225 & \underline{0.897} & 0.947 & 0.957 & \textbf{0.310} & 110.6 & 61.4 & \underline{5.3} & \textbf{0.9} \\
cif-2016 & \textbf{0.918} & \underline{0.986} & 1.025 & 1.289 & 0.0